In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import time
import os

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
path = r'D:\РЭШ\Research\PostThesis\data'

In [4]:
path_var = path + r'\var'
dir_list = os.listdir(path_var)

files = []
for file in dir_list:
    if file.find('_var_real_ruo.xlsx')>0:
        files.append(file)
files

['bankdata_2017-2020_y_var_real_ruo.xlsx',
 'bankdata_201709-202109_q_var_real_ruo.xlsx']

In [5]:
i=0
df = pd.read_excel(path_var + r'\\' + files[i])

In [6]:
df.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'regnum', 'dt_year',
       'dt_month', 'quarter', 'Name', 'Total_assets', 'Net_worth',
       'Net_income', 'Loans', 'Admin_expenses', 'Rel_Admin_expenses',
       'Securities', 'Cash', 'Liabilities', 'NIE', 'NII', 'II_Loans',
       'IE_Deposits', 'Leverage', 'Deposits', 'Pure_deposits', 'Sales',
       'Safe_funds', 'Safe_funds_income', 'Safe_rev', 'PoL', 'CoF', 'ruo',
       'key_rate'],
      dtype='object')

# Winsorizing and Summary statistics
Note that the following variables are used for markup estimation:
* NIE (Non-interest expenses)
* NII (Non-interest income)
* Loans (Total loans)
* Deposits (Total finacial liabilities, including deposits, interbank loans, bonds etc)
* Securities (Total securities holdings)
* II_Loans (Interest income on loans)
* IE_Deposits (Interest expenses on deposits)

For HHI also used:
* Sales (Total interest income + Comission income)

Save to the folder ```var_wins```.

In [7]:
def Winsorize(df, q_min, q_max):
    """ 
    The function does Winsorizing the necessary columns for the DataFrame with correspoding quantiles
    Quantiles are from 0 to 1.
    
    Returns Winsorized df and df_stat with summary statistics of Winsorized columns.
    """
    col = 'Total_assets'
    df_stat = pd.DataFrame({'Variable': [col], 
                            'Min': [df[col].min()], 
                            'q = '+str(q_min): [df[col].quantile(q_min)], 
                            'Median': [df[col].median()], 
                            'Mean': [df[col].mean()], 
                            'q = '+str(q_max): [df[col].quantile(q_max)], 
                            'Max': [df[col].max()]})
    # add summary statistics
    for col in ['Net_worth', 'Net_income']:
        df_temp = pd.DataFrame({'Variable': [col], 
                            'Min': [df[col].min()], 
                            'q = '+str(q_min): [df[col].quantile(q_min)], 
                            'Median': [df[col].median()], 
                            'Mean': [df[col].mean()], 
                            'q = '+str(q_max): [df[col].quantile(q_max)], 
                            'Max': [df[col].max()]})
        df_stat = pd.concat([df_stat, df_temp], ignore_index = True)
    
    
    for col in ['NIE','NII', 'Loans', 'Deposits', 'Securities', 'II_Loans', 'IE_Deposits', 'Sales']:
        # add column devided to Total assets
        df[col+'_TA'] = df[col]/df['Total_assets']

        # calculate quantiles
        q_max_value = df[col+'_TA'].quantile(q_max)
        q_min_value = df[col+'_TA'].quantile(q_min)

        # add summary statistics
        df_temp = pd.DataFrame({'Variable': [col, col+'/ Total_assets'], 
                            'Min': [df[col].min(), df[col+'_TA'].min()], 
                            'q = '+str(q_min): [df[col].quantile(q_min), df[col+'_TA'].quantile(q_min)], 
                            'Median': [df[col].median(), df[col+'_TA'].median()], 
                            'Mean': [df[col].mean(), df[col+'_TA'].mean()], 
                            'q = '+str(q_max): [df[col].quantile(q_max), df[col+'_TA'].quantile(q_max)], 
                            'Max': [df[col].max(), df[col+'_TA'].max()]})
        df_stat = pd.concat([df_stat, df_temp], ignore_index = True)

        # replace the tales with corresponding quantiles
        for index in df.loc[df[col+'_TA']>q_max_value].index:
            df.loc[index, col+'_TA'] = q_max_value
            df.loc[index, col] = df.loc[index, col+'_TA'] * df.loc[index, 'Total_assets']

        for index in df.loc[df[col+'_TA']<q_min_value].index:
            df.loc[index, col+'_TA'] = q_min_value
            df.loc[index, col] = df.loc[index, col+'_TA'] * df.loc[index, 'Total_assets']
        
    # update dependent columns: PoL (Price of Loans), CoF (Cost of Funds)
    #df.drop(columns = ['PoL', 'CoF'])
    df['PoL'] = df['II_Loans'] / df['Loans']
    df['CoF'] = df['IE_Deposits'] / df['Deposits']
    
    return df, df_stat

In [8]:
path_var = path + r'\var'
path_to = path + r'\var'
path_stat = path
q_min = 0.01
q_max = 0.99

for file in files:
    df = pd.read_excel(path_var + r'\\' + file)
    df_output, df_stat = Winsorize(df, q_min, q_max)
    df_stat.to_excel(path_stat + r'\\' + file[:file.find('.')] + '_stat.xlsx')
    df_output.to_excel(path_to + r'\\' + file[:file.find('.')] + '_wins.xlsx')
    #df_output.to_csv(path_to + r'\\' + file[:file.find('.')] + '_wins.csv', encoding='utf-8')

## NOT NEEDED
#### Final Clean up
Delete Toyota bank (register number 3470) and delete unnecessary columns. Save to the folder ```vars```. This action is not needed since Toyota bank doesn't have any securities and therefore will drop out from markup estimation.

In [9]:
path_var = path + r'\var'
dir_list = os.listdir(path_var)

files = []
for file in dir_list:
    if file.find('_ruo_wins.xlsx')>0:
        files.append(file)
files

['bankdata_2017-2020_y_var_real_ruo_wins.xlsx',
 'bankdata_201709-202109_q_var_real_ruo_wins.xlsx']

In [10]:
def delete_unnamed_cols(df):
    columns_delete = []
    for col in df.columns:
        if col.find('Unnamed:')>=0:
            columns_delete.append(col)
        if col.find('_TA')>0:
            columns_delete.append(col)
    print('Columns deleted:', columns_delete)
    return df.drop(columns=columns_delete)

In [11]:
path_var = path + r'\var'
path_to = path + r'\var'

for file in files:
    df = pd.read_excel(path_var + r'\\' + file)
    df = df.loc[df['regnum']!=3470] # delete Toyota bank
    df = delete_unnamed_cols(df) # delete redundant columns
    df.to_excel(path_to + r'\\' + file[:file.find('.')] + '_clean.xlsx')
    #df.to_csv(path_to + r'\\' + file[:file.find('.')] + '_clean.csv', encoding='utf-8')

Columns deleted: ['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'NIE_TA', 'NII_TA', 'Loans_TA', 'Deposits_TA', 'Securities_TA', 'II_Loans_TA', 'IE_Deposits_TA', 'Sales_TA']
Columns deleted: ['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'NIE_TA', 'NII_TA', 'Loans_TA', 'Deposits_TA', 'Securities_TA', 'II_Loans_TA', 'IE_Deposits_TA', 'Sales_TA']
